In [ ]:
import yfinance as yf
import pandas as pd
import requests



def get_risk_free_rate_from_fred(api_key):
    url = "https://api.stlouisfed.org/fred/series/observations"
    params = {
        'series_id': 'DGS3MO',  # 3-Month Treasury Bill rate
        'api_key': api_key,
        'file_type': 'json',
        'limit': 1,
        'sort_order': 'desc',  # Get the latest data
    }

    response = requests.get(url, params=params)
    data = response.json()
    if response.status_code == 200 and 'observations' in data and len(data['observations']) > 0:
        latest_rate = data['observations'][0]['value']
        return float(latest_rate) / 100  # Convert percentage to decimal
    else:
        raise Exception("Could not fetch the risk-free rate from FRED.")




# Function to fetch historical data using yfinance
def get_historical_data(ticker, interval):
    stock = yf.Ticker(ticker)
    df = stock.history(period="max", interval=interval)
    df.index = df.index.tz_localize(None)  # Convert index to be timezone-naive
    return df

# Function to calculate Sharpe Ratio
def calculate_sharpe_ratio(historical_data, risk_free_rate, period):
    # Calculate returns based on the specified period
    if period == '1d':  # Daily returns
        daily_returns = historical_data['Close'].pct_change().dropna()
    elif period == '1wk':  # Weekly returns
        daily_returns = historical_data['Close'].resample('W').ffill().pct_change().dropna()
    elif period == '1mo':  # Monthly returns
        daily_returns = historical_data['Close'].resample('M').ffill().pct_change().dropna()
    # You can add more elif blocks for different periods if needed

    mean_daily_returns = daily_returns.mean()
    std_dev_daily_returns = daily_returns.std()

    # Adjust the annualization factor based on the period
    if period == '1d':
        annualization_factor = 252
    elif period == '1wk':
        annualization_factor = 52
    elif period == '1mo':
        annualization_factor = 12
    # You can add more elif blocks for different periods if needed

    annualized_mean_return = mean_daily_returns * annualization_factor
    annualized_std_dev = std_dev_daily_returns * (annualization_factor ** 0.5)

    sharpe_ratio = (annualized_mean_return - risk_free_rate) / annualized_std_dev
    return sharpe_ratio

# Function to process tickers and calculate Sharpe Ratios
def process_tickers(tickers, risk_free_rate, period):
    results = []
    for ticker in tickers:
        ticker = ticker.strip().upper()  # Clean up ticker string
        try:
            historical_data = get_historical_data(ticker, period)
            sharpe_ratio = calculate_sharpe_ratio(historical_data, risk_free_rate, period)
            results.append((ticker, sharpe_ratio))
        except Exception as e:
            results.append((ticker, f"Error: {e}"))
    return results

# Main function to run the script
def main():
    # Get user input for tickers
    tickers = input("Enter the tickers separated by a comma (e.g., BA, BAC): ").split(',')

    # Get user input for the period
    period = input("Enter the return period (e.g., 1d for daily, 1wk for weekly, 1mo for monthly): ").strip()

    fred_api_key = 'eb09a63015e1cac7136fe16731853aee'  # Replace with your actual FRED API key
    risk_free_rate = get_risk_free_rate_from_fred(fred_api_key)
    print(f"Current risk-free rate: {risk_free_rate}")

    tickers = ['BA', 'BAC']

    if risk_free_rate is None:
        print("Could not obtain the risk-free rate. Exiting the script.")
        return

    results = process_tickers(tickers, risk_free_rate, period)
    results_df = pd.DataFrame(results, columns=['Ticker', 'Sharpe Ratio'])
    print(results_df)
    # Optionally, save results to CSV
    results_df.to_csv('sharpe_ratios.csv', index=False)

if __name__ == "__main__":
    main()


Enter the tickers separated by a comma (e.g., BA, BAC): BAC
Enter the return period (e.g., 1d for daily, 1wk for weekly, 1mo for monthly): 1d
Current risk-free rate: 0.0547
  Ticker  Sharpe Ratio
0     BA      0.337701
1    BAC      0.205654
